# Welcome to My colab ;)

#### [Documentation-](https://docs.google.com/document/d/1j-_7QNILUBYJq0h3u99sEotJIntOJAMc5mpf5_RJyaA/edit?usp=sharing)

       -Kanav Anand

## All packages are imported in this cell

In [0]:
from google.colab import files
import time,sys,warnings,glob,random,cv2,base64,json,csv,os
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import graph_util
import subprocess
import tempfile
import time,sys,warnings,glob,random,cv2,base64,json,csv,os
import numpy as np
from tensorflow.python.framework import graph_util

### Run this cell to upload a zip file containing


1.   .pb file 
2.   .Meta file(inclused all the checkpoint.)    

In [4]:
upload=files.upload()

Saving CD_Check_k.zip to CD_Check_k.zip


Run this cell to check the te name of the files present ,**make sure zip file you uploaded should be there**

In [36]:
ls

CD_Check_k.zip  FF_Check_k/     FF_OD_RGB_squeeze_13Oct.tflite
datalab/        FF_Check_k.zip


Get the name of the file from above zip and extract it. 

In [37]:
!unzip CD_Check_k.zip

Archive:  CD_Check_k.zip
   creating: CD_Check_k/
  inflating: CD_Check_k/CD_L_OD_19_RGB_squeeze_128_9may.pb  
  inflating: CD_Check_k/checkpoint   
  inflating: CD_Check_k/filename_CD.ckpt.data-00000-of-00001  
  inflating: CD_Check_k/filename_CD.ckpt.index  
  inflating: CD_Check_k/filename_CD.ckpt.meta  


**As you can clearly see the all the files are extracted in the folder named "FF_Check_k" which is the name of your zip. **

So our current working directory is the  above one:

# These are the inputs careflully fill according to the need.

In [0]:
##input shape
img_size_x = 256
img_size=_y=256

##Directory 
direc ="CD_Check_k/"
#####frozen moedl pb:

frozen_model_pb = 'CD_L_OD_19_RGB_squeeze_128_9may.pb'
#####checkpoint model with ckpt extension:

checkpoint_path="CD_Check_k/filename_CD.ckpt"
###### output file name:

###input tensor name



In [0]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        

        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
        
    return graph
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)



# This is the main conversion section of this code.
Here you can modify the name of the tensors according to your need.

In [0]:

def conversion(frozen_model,image_size_x,image_size_y,path,output_name,num_channels=3,input_tensor='prefix/input_node:0',output_tensor='prefix/output_node:0',download=True):
  e=time.time()
  out_name=frozen_model_pb[:-3]
  CD_graph = load_graph(direc+frozen_model_pb)  
  CD_input = CD_graph.get_tensor_by_name(input_tensor)
  CD_output = CD_graph.get_tensor_by_name(output_tensor)
  output_single = tf.identity(tf.nn.softmax(CD_output, axis=1), name="output_single")
  x_single = tf.placeholder(dtype=tf.float32, shape=[1,img_size,img_size,num_channels],
                                  name="input_node")
  tf.contrib.lite.tempfile = tempfile
  tf.contrib.lite.subprocess = subprocess
  print('************************CONVERTING TO THE TFLITE************************')
  with tf.Session(graph=CD_graph) as sess:
    saver = tf.train.import_meta_graph(checkpoint_path + ".meta", import_scope=None)
    saver.restore(sess, checkpoint_path)
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph = graph_util.convert_variables_to_constants(sess, graph_def, ["output_single"])
    tflite_model = tf.contrib.lite.toco_convert(output_graph, input_tensors=[x_single], output_tensors=[output_single])
    with open('./'+out_name+'.tflite', "wb") as f:
      f.write(tflite_model)
  print('************************SUCCESSFULLY CONVERTED ************************')
  if (download):
    files.download(out_name+'.tflite')
    print('Download successfully')



In [43]:
def main():
  conversion(frozen_model_pb,img_size,direc,output_name=out_name,download=True)
  print("Done")

if __name__ == "__main__":
    s = time.time()
    main()
    e = time.time()
    print('time take :',e-s)
    print('The file you uploaded (.PB) size:',file_size(direc+frozen_model_pb))    
    print('The Tflite size :',file_size(out_name+'.tflite'))
    

************************CONVERTING TO THE TFLITE************************
INFO:tensorflow:Restoring parameters from CD_Check_k/filename_CD.ckpt
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
************************SUCCESSFULLY CONVERTED ************************
Download successfully
Done
time take : 6.509053707122803
The file you uploaded (.PB) size: 1.2 MB
The Tflite size : 1.2 MB
